In [ ]:
!pip install "protobuf==3.20.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tens

In [ ]:
padded_path = "/kaggle/input/arabic-diacritics/keras/default/1/padded.pkl"
val_path = "/kaggle/input/val-2/padded_val.pkl"

diacritic2id_path = "/kaggle/input/arabic-diacritics/keras/default/1/diacritic2id.json"
idx2char_path = "/kaggle/input/arabic-diacritics/keras/default/1/idx2char.json"
char2idx_path = "/kaggle/input/arabic-diacritics/keras/default/1/char2idx.json"

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Input
import json
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

2025-11-30 07:04:42.861469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764486283.070395      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764486283.126676      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
with open(diacritic2id_path, 'r', encoding='utf-8') as f:
    label2id = json.load(f)

id2label = {v: k for k, v in label2id.items()}

with open(char2idx_path, 'r', encoding='utf-8') as f:
    char2idx = json.load(f)

with open(idx2char_path, 'r', encoding='utf-8') as f:
    idx2char = json.load(f)

In [ ]:
MAX_SEQ_LEN = 807
CHAR_EMBED_DIM = 256
BATCH_SIZE = 32
EPOCHS = 10
VOCAB_SIZE = len(char2idx)
NUM_CLASSES = len(label2id)
DIRTY = False
DIACRITICS_PATTERN = re.compile(r'[\u064B-\u0652]')

In [ ]:
def load_val_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    X_txt = []
    y_label = []

    for text_seq, label_seq in zip(X_raw, y_raw):

        x_ids = [char2idx[c] for c in text_seq]
        y_ids = [label2id[t] for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)



In [ ]:
def load_data_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        x_ids = [char2idx[c] for c in text_seq]
        y_ids = [label2id[t] for t in label_seq]

        X.append(x_ids)
        y.append(y_ids)

    return np.array(X), np.array(y)

In [ ]:
def load_text_pickle(file_path):
    with open(file_path, 'rb') as f:
        X_raw, y_raw = pickle.load(f)
    X = []
    y = []

    for text_seq, label_seq in zip(X_raw, y_raw):
        for i in range(len(text_seq)):
            if '(' == text_seq[i] and label_seq[i] != '':
                print(label_seq[i])
                return
        X.append(text_seq)
        y.append(label_seq)

    return X, y

In [ ]:
X, y = load_data_pickle(padded_path)

In [ ]:
raw_X, raw_y = load_text_pickle(padded_path)

In [ ]:
raw_X_val, raw_y_val = load_text_pickle(val_path)

In [ ]:
X_val, y_val = load_val_pickle(val_path)

In [ ]:
if DIRTY:
    X = pad_sequences(X, maxlen=500, padding='post', truncating='post')
    y = pad_sequences(y, maxlen=500, padding='post', truncating='post')
    MAX_SEQ_LEN = 500

padding_mask = (X == 0)
y[padding_mask] = -1

padding_mask = (X_val == 0)
y_val[padding_mask] = -1

In [ ]:
class MaskedSparseCategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)

        mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)

        y_true_safe = tf.where(tf.equal(y_true, -1), tf.zeros_like(y_true), y_true)

        loss = tf.keras.losses.sparse_categorical_crossentropy(
            y_true_safe, y_pred, from_logits=False
        )

        p_t = tf.math.exp(-loss)
        focal_loss = tf.math.pow(1.0 - p_t, self.gamma) * loss

        masked_focal_loss = focal_loss * mask

        return tf.reduce_sum(masked_focal_loss) / (tf.reduce_sum(mask) + 1e-7)

In [ ]:
def masked_accuracy(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)

    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true = tf.cast(y_true, y_pred_classes.dtype)

    matches = tf.cast(tf.equal(y_true, y_pred_classes), tf.float32)

    matches *= mask

    return tf.reduce_sum(matches) / (tf.reduce_sum(mask) + 1e-7)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Bidirectional, LSTM

CHAR_EMBED_DIM = 256
BATCH_SIZE = 32
VOCAB_SIZE = len(char2idx)
NUM_CLASSES = len(label2id)

input_chars = Input(shape=(None,), dtype='int32', name='char_input')

target_embedding_layer = Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=CHAR_EMBED_DIM,
    # mask_zero=True,
    name='char_embedding'
)
x = target_embedding_layer(input_chars)

x = Bidirectional(LSTM(256, return_sequences=True, name='lstm_1'))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Bidirectional(LSTM(128, return_sequences=True, name='lstm_2'))(x)
x = tf.keras.layers.Dropout(0.3)(x)

output = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model = tf.keras.Model(inputs=input_chars, outputs=output)

model.compile(
    optimizer='adam',
    loss=MaskedSparseCategoricalFocalLoss(gamma=2.0),
    metrics=[masked_accuracy]
)

model.summary()

history = model.fit(
    X, y,
    batch_size=BATCH_SIZE,
    epochs=5,
    validation_data=(X_val, y_val)
)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ char_input (InputLayer)         │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ char_embedding (Embedding)      │ (None, None, 256)      │        13,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, None, 512)      │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, None, 256)      │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, None, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, None, 15)       │         3,855 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,724,687 (6.58 MB)

 Trainable params: 1,724,687 (6.58 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
2508/2508 ━━━━━━━━━━━━━━━━━━━━ 492s 194ms/step - loss: 0.3088 - masked_accuracy: 0.8132 - val_loss: 0.0571 - val_masked_accuracy: 0.9533
Epoch 2/5
2508/2508 ━━━━━━━━━━━━━━━━━━━━ 487s 194ms/step - loss: 0.0628 - masked_accuracy: 0.9491 - val_loss: 0.0416 - val_masked_accuracy: 0.9644
Epoch 3/5
2508/2508 ━━━━━━━━━━━━━━━━━━━━ 487s 194ms/step - loss: 0.0474 - masked_accuracy: 0.9602 - val_loss: 0.0364 - val_masked_accuracy: 0.9687
Epoch 4/5
2508/2508 ━━━━━━━━━━━━━━━━━━━━ 487s 194ms/step - loss: 0.0403 - masked_accuracy: 0.9652 - val_loss: 0.0328 - val_masked_accuracy: 0.9714
Epoch 5/5
2508/2508 ━━━━━━━━━━━━━━━━━━━━ 487s 194ms/step - loss: 0.0362 - masked_accuracy: 0.9682 - val_loss: 0.0306 - val_masked_accuracy: 0.9734


In [ ]:
import pickle
import tensorflow as tf

model.save('/kaggle/working/diacritization_model.keras')

packaging = {
    'char2id': char2idx,
    'id2label': id2label
}

with open('/kaggle/working/mappings.pkl', 'wb') as f:
    pickle.dump(packaging, f)

print("Model and mappings saved successfully.")

Model and mappings saved successfully.


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

print("Generating predictions...")
y_pred = model.predict(X_val, batch_size=32)
y_pred_classes = np.argmax(y_pred, axis=-1)

mask = X_val.flatten() != 0
y_true_clean = y_val.flatten()[mask]
y_pred_clean = y_pred_classes.flatten()[mask]

target_names = [k for k, v in sorted(label2id.items(), key=lambda item: item[1])]

print(classification_report(y_true_clean, y_pred_clean, target_names=target_names))

Generating predictions...
123/123 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step
              precision    recall  f1-score   support

           َ       0.96      0.97      0.97    144327
           ً       0.92      0.94      0.93      2997
           ُ       0.93      0.90      0.92     38932
           ٌ       0.85      0.85      0.85      2865
           ِ       0.96      0.96      0.96     61952
           ٍ       0.92      0.90      0.91      4372
           ْ       0.98      0.99      0.98     60276
           ّ       0.84      0.17      0.28       254
          َّ       0.95      0.95      0.95     14618
          ًّ       0.93      0.95      0.94       189
          ُّ       0.91      0.89      0.90      2185
          ٌّ       0.80      0.87      0.83       239
          ِّ       0.92      0.88      0.90      3156
          ٍّ       0.85      0.85      0.85       300
                   1.00      1.00      1.00    198025

    accuracy                           0.97    534687
   macro av

In [ ]:
import numpy as np

for test_index in range(len(raw_X_val[:10])):

    raw_text = raw_X_val[test_index]
    input_seq = X_val[test_index]

    input_ready = np.expand_dims(input_seq, axis=0)
    y_pred = model.predict(input_ready)
    y_pred_classes = np.argmax(y_pred, axis=-1)[0]

    pred_result = ''
    act_result = ''

    for j in range(len(raw_text)):
        char = raw_text[j]
        if char == '<PAD>':
            break
        pred_id = y_pred_classes[j]

        predicted_diacritic = id2label[pred_id]

        pred_result += char + predicted_diacritic
        act_result += char + raw_y_val[test_index][j]

    print("True:   ", act_result)
    print("Predicted:   ", pred_result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
True:    الشَّهَادَةِ ظَاهِرَةً ، وَبِحَقٍّ بَيِّنٍ تَضْعُفُ التُّهْمَةُ ، وَهُوَ الْفَرْقُ بَيْنَهُ وَبَيْنَ الشَّهَادَةِ ، وَعَنْ أَصْبَغَ الْجَوَازُ فِي الْوَلَدِ وَالزَّوْجَةِ وَالْأَخِ وَالْمُكَاتَبِ وَالْمُدَبَّرِ وَالْمِدْيَانِ إنْ كَانَ مِنْ أَهْلِ الْقِيَامِ بِالْحَقِّ ، وَصَحَّ الْحُكْمُ ، وَقَدْ يَحْكُمُ لِلْخَلِيفَةِ ، وَهُوَ فَوْقَهُ ، وَتُهْمَتُهُ أَقْوَى ، وَلَا يَنْبَغِي لَهُ الْقَضَاءُ بَيْنَ أَحَدٍ مِنْ عَشِيرَتِهِ وَخَصْمِهِ ، وَإِنْ رَضِيَ الْخَصْمُ بِخِلَافِ رَجُلَيْنِ رَضِيَا بِحُكْمِ رَجُلٍ أَجْنَبِيٍّ فَيَنْفُذُ ذَلِكَ عَلَيْهِمَا ، وَلَا يَقْضِي بَيْنَهُ وَبَيْنَ غَيْرِهِ ، وَإِنْ رَضِيَ الْخَصْمُ بِذَلِكَ فَإِنْ فَعَلَ فَيُشْهِدُ عَلَى رِضَاهُ ، وَيَجْتَهِدُ فِي الْحَقِّ فَإِنْ قَضَى لِنَفْسِهِ أَوْ لِمَنْ يَمْتَنِعُ قَضَاؤُهُ لَهُ فَلْيَذْكُرْ الْقِصَّةَ كُلَّهَا ، وَرَضِيَ خَصْمِهِ ، وَشَهَادَةَ مَنْ شَهِدَ بِرِضَى الْخَصْمِ .
Predicted:    الشَّهَادَةِ ظَاهِرَةٌ ، وَبِحَقٍّ بَيْنَ تَضَعُّفِ التُّهْمَةَ ، وَهُوَ الْفَرْق